<a href="https://colab.research.google.com/github/rubygitflow/Glossary-extraction/blob/main/University_of_Artificial_Intelligence_Glossary_extraction_Outside_Test_2020_v_0_12_12_11_min.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Glossary extraction. 

### The part-of-speech way. 
### Multiple Inputs: Lexemes, parts of speech, morphological features, alphabet.

https://www.pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/ 

**The research project**

В текущей версии: 
* реализован минималистичный вариант для обучения нейронной сети без тщательных проверок и для её тестирования на текстах из внешних файловых источников.

# **Service data**

In [ ]:
!nvidia-smi

Ссылки на файлы данных:

База_словарей_train_6.docx : https://drive.google.com/file/d/13QKXApDf8q8R_BYviX1J80BCJXUOlZUw/view?usp=sharing

База_словарей_test_6.docx : https://drive.google.com/file/d/19a1rOu6NuAb-Y_HrHhN7Ssg_ZvxTXHhH/view?usp=sharing

База_словарей_train_6.xml : https://drive.google.com/file/d/16lNQFww1QyYRxr8S4OsA2THypqxLQOra/view?usp=sharing

База_словарей_test_6.xml : https://drive.google.com/file/d/1-9fG7sdiZnjkarvitPYgX9PFt95eErPi/view?usp=sharing


In [ ]:
color_meaning = {'00ffff':'термин', 'ffff00':'связка', '00ff00':'определение', 'd3d3d3':'расшифровка', 
                 'ff00ff':'дубль-термин', 'ff0000':'дубль-связка', '808000':'дубль-определение', '0000ff':'дубль-расшифровка',
                 'cyan':'термин','yellow':'связка','green':'определение', 'lightgray':'расшифровка',	                 
                 'magenta':'дубль-термин', 'red':'дубль-связка','darkyellow':'дубль-определение', 'blue':'дубль-расшифровка'}

                 #создаем словарь соответствий цвет-значение   

In [ ]:
color_dic = {'термин':0,'связка':1,'определение':2,'расшифровка':3,
			       'дубль-термин':4,'дубль-связка':5,'дубль-определение':6,'дубль-расшифровка':7}
dic_color = {value:key for key, value in color_dic.items()}

zero_symbol = 'o'

colors_count = len(color_dic) + 1 # количество уникальных тегов (не забываем про тег отсутствия класса для разметки)

skip_amount = colors_count  # кол-во пропускаемых строк под легенду при считывании данных (здесь учитывается, что есть пусая строка, в списке тегов последнее значение используется под отсутствующий класс для разметки)

In [ ]:
colors_count

In [ ]:
# названия необходимых тегов для извлекаемых категорий
r = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}r'
rpr = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}rPr'
t = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}t'
shd = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}shd'
fill = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}fill'
highlight = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}highlight'
val = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}val'

In [ ]:
''' 
  Словарь для цветовой разметки разобранного текста
'''
class color:
  GREY    = '\33[90m'
  RED2    = '\33[91m'
  GREEN2  = '\33[92m'
  YELLOW2 = '\33[93m'
  BLUE2   = '\33[94m'
  VIOLET2 = '\33[95m'
  LI_CYAN = '\33[96m'
  WHITE2  = '\33[97m'

  BLACK  = '\33[30m'
  RED    = '\33[31m'
  GREEN  = '\33[32m'
  YELLOW = '\33[33m'
  BLUE   = '\33[34m'
  VIOLET = '\33[35m'
  CYAN   = '\33[36m'
  WHITE  = '\33[37m'

  GREYBG    = '\33[100m'
  REDBG2    = '\33[101m'
  GREENBG2  = '\33[102m'
  YELLOWBG2 = '\33[103m'
  BLUEBG2   = '\33[104m'
  VIOLETBG2 = '\33[105m'
  LI_CYANBG = '\33[106m'
  WHITEBG2  = '\33[107m'

  BLACKBG  = '\33[40m'
  REDBG    = '\33[41m'
  GREENBG  = '\33[42m'
  YELLOWBG = '\33[43m'
  BLUEBG   = '\33[44m'
  VIOLETBG = '\33[45m'
  CYANBG   = '\33[46m'
  WHITEBG  = '\33[47m'


  BOLD 		  = '\33[1m'
  ITALIC 	  = '\33[3m'
  UNDERLINE = '\33[4m'

  BLINK    = '\33[5m'
  BLINK2   = '\33[6m'
  SELECTED = '\33[7m'
  END 		 = '\33[0m'
  
dic_colored_items = {0:color.LI_CYANBG+color.BLACK, 1:color.YELLOWBG2+color.BLACK, 2:color.GREENBG2+color.BLACK, 3:color.WHITEBG+color.BLACK,
			               4:color.VIOLETBG2+color.WHITE2, 5:color.REDBG2+color.WHITE2, 6:color.YELLOWBG+color.BLACK, 7:color.BLUEBG+color.WHITE2}

## Connecting libraries

In [ ]:
!pip install pymorphy2

In [ ]:
!pip install polyglot
# https://github.com/aboSamoor/polyglot/issues/152
!pip install PyICU
!pip install pycld2
!pip install morfessor

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model # Импортируем модели keras
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, concatenate, Activation, MaxPooling1D, Conv1D, concatenate  # Импортируем стандартные слои keras
from tensorflow.keras.layers import Dropout, BatchNormalization, Conv2DTranspose, Lambda, LSTM, GRU, Bidirectional, GlobalMaxPool1D # Импортируем стандартные слои keras
from tensorflow.keras.layers import SpatialDropout1D 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras import backend as K # Импортируем модуль backend keras'а
from tensorflow.keras.optimizers import Adam, RMSprop, Adadelta
from tensorflow.keras import utils # Импортируем модуль utils библиотеки tensorflow.keras для получения OHE-представления
from tensorflow.keras.metrics import AUC
from tensorflow.keras.models import load_model

from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt 
import collections
%matplotlib inline
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report # статистика точности предсказания по классам
import random
import datetime
import time

import xml.etree.ElementTree as ET

import re # https://habr.com/ru/post/349860/
import pymorphy2 # https://pymorphy2.readthedocs.io/en/latest/user/guide.html
import polyglot # https://polyglot.readthedocs.io/en/latest/
from polyglot.detect import Detector # https://polyglot.readthedocs.io/en/latest/Detection.html
from textblob import TextBlob # https://textblob.readthedocs.io/en/latest/quickstart.html
# import nltk
# from nltk import pos_tag # https://stackoverflow.com/questions/15388831/what-are-all-possible-pos-tags-of-nltk
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

import os # Импортируем библиотеку для работы с файловой системой
from google.colab import files # Импортируем Модуль для работы с файлами
from google.colab import drive # Подключаем гугл-диск

from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

In [ ]:
drive.mount('/content/drive')

In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
all_tags_together = ['o термин связка определение расшифровка дубль-термин дубль-связка дубль-определение дубль-расшифровка']
tag_tokenizer = Tokenizer(filters=' ')
tag_tokenizer.fit_on_texts(all_tags_together)
tag_tokenizer.index_word

In [ ]:
# prepare color dictionary
tag_tokenizer_dic_color = {}
tag_tokenizer_dic_color[1] = color.END
for i in range(2,10):
  tag_tokenizer_dic_color[i] = dic_colored_items[color_dic[tag_tokenizer.index_word[i]]]
tag_tokenizer_dic_color

## Library of local Functions

In [ ]:
'''
  Сервиcная функция:
  Лемматизация для английского языка
  Функция предполагает разбор именно слов (без пробелов внутри текста - Это важно), 
  но лучшим оказался алгоритм лемматизации предложений
'''
# https://webdevblog.ru/podhody-lemmatizacii-s-primerami-v-python/
def textblob_lemmatize(blob):
  sent = TextBlob(blob)
  tag_dict = {"J": 'a', 
              "N": 'n', 
              "V": 'v', 
              "R": 'r'}
  words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
  lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
  res = "".join(lemmatized_list)
  return res if res!='' else blob

In [ ]:
'''
  Сервиcная функция:
  Извлечение тега для лексемы на английском
  https://www.geeksforgeeks.org/python-part-of-speech-tagging-using-textblob/
  CC coordinating conjunction
  CD cardinal digit
  DT determiner
  EX existential there (like: “there is” … think of it like “there exists”)
  FW foreign word
  IN preposition/subordinating conjunction
  JJ adjective ‘big’
  JJR adjective, comparative ‘bigger’
  JJS adjective, superlative ‘biggest’
  LS list marker 1)
  MD modal could, will
  NN noun, singular ‘desk’
  NNS noun plural ‘desks’
  NNP proper noun, singular ‘Harrison’
  NNPS proper noun, plural ‘Americans’
  PDT predeterminer ‘all the kids’
  POS possessive ending parent‘s
  PRP personal pronoun I, he, she
  PRP$ possessive pronoun my, his, hers
  RB adverb very, silently,
  RBR adverb, comparative better
  RBS adverb, superlative best
  RP particle give up
  TO to go ‘to‘ the store.
  UH interjection errrrrrrrm
  VB verb, base form take
  VBD verb, past tense took
  VBG verb, gerund/present participle taking
  VBN verb, past participle taken
  VBP verb, sing. present, non-3d take
  VBZ verb, 3rd person sing. present takes
  WDT wh-determiner which
  WP wh-pronoun who, what
  WP$ possessive wh-pronoun whose
  WRB wh-abverb where, when

  https://pymorphy2.readthedocs.io/en/latest/user/grammemes.html#russian-numbers
  Граммема	Значение	          Примеры
  sing	    единственное число	хомяк, говорит
  plur	    множественное число	хомяки, говорят
'''
def textblob_tags(blob):
  # print('textblob_tags:',blob, TextBlob(blob).tags)
  try:
    pos = TextBlob(blob).tags[0][1]
    if pos in ['NN', 'NNP']:
      numbers = 'sing'
    elif pos in ['NNS', 'NNPS']:
      numbers = 'plur'
    else:
      numbers = 'None'
  except:
    pos = 'NN'
    numbers = 'sing'

  return pos, numbers

In [ ]:
'''
  Сервиcная функция:
  Назначение граммемы
  https://pymorphy2.readthedocs.io/en/latest/user/grammemes.html
'''
def define_morphy( 
  pos = 'None', animacy = 'None', aspect = 'None',
  case = 'None', gender = 'None', involvement = 'None',
  mood = 'None', number = 'None', person = 'None',
  tense = 'None', transitivity = 'None', voice = 'None'
): 
  return [str(pos), str(animacy), str(aspect), str(case), str(gender), str(involvement), 
          str(mood), str(number), str(person), str(tense), str(transitivity), str(voice)]



In [ ]:
'''
  Сервиcная функция:
  Извлечение морфологических признаков лексемы 
'''
def morphyus(blob):
  # 1. язык: долюно быть - sign, en, ru, el. Другие пока не расматриваются
  # 2. С заглавной ли буквы написание слова
  # итерации по
    # 3. слово в нормальной форме
    # 4. часть речи
    # 5. всё остальное

  draw_in = lambda a: ''.join(a.split())

  
  morphy = []  
  tags = morph.parse(blob)
  # print('tags',tags)
  try:
    detector = Detector(blob)
    language = detector.language.code
    # print('language',language)
  except:
    lng_lst = str(tags[0].tag).split(',')
    # print('lng_lst',lng_lst)
    if lng_lst[0]=='UNKN':
      morphy.append('sign')
      morphy.append('true' if blob[0].isupper() else 'false')
      morphy.append(textblob_lemmatize(blob).lower())
      pos, number = textblob_tags(blob)
      morphy += define_morphy(pos = pos, number = number)         
      return morphy
    elif lng_lst[0]=='LATN':
      morphy.append('en')
      morphy.append('true' if blob[0].isupper() else 'false')
      morphy.append(textblob_lemmatize(blob).lower())
      pos, number = textblob_tags(blob)
      morphy += define_morphy(pos = pos, number = number)  
      return morphy
    elif lng_lst[0]=='NUMB':
      morphy.append('sign')
      morphy.append('false')
      morphy.append(draw_in(tags[0].normal_form)) # allways .lower()
      morphy += define_morphy(pos = 'NUMB', number = lng_lst[1])  
      return morphy
    elif lng_lst[0]=='ROMN':
      morphy.append('en')
      morphy.append('true' if blob[0].isupper() else 'false')
      morphy.append(textblob_lemmatize(blob).lower())
      pos, number = textblob_tags(blob)
      morphy += define_morphy(pos = pos, number = number)  
      # print 'ROMN' morfology
      morphy.append(draw_in(tags[0].normal_form)) # allways .lower()
      morphy += define_morphy(pos = 'ROMN')
      return morphy
    elif lng_lst[0]=='PNCT':
      morphy.append('sign')
      morphy.append('false')
      morphy.append(draw_in(tags[0].normal_form)) # allways .lower()
      morphy += define_morphy(pos = 'PNCT')  
      return morphy
    elif lng_lst[0] in ['NOUN','ADJF','ADJS','COMP','VERB','INFN','PRTF','PRTS','GRND','NUMR','ADVB','NPRO','PRED','PREP','CONJ','PRCL','INTJ']:
      language = 'ru'
    else:
      language = lng_lst[0]

  # print('language:',language, '; blob:', blob)
  if language in ['bg', 'uk', 'mn', 'ab', 'ba', 'be', 'ky', 'mk', 'ro', 'ru', 'sr', 'tk', 'tt', 'ug', 'uz', 'ug']: 
  # на самом деле плохое условие. Надо менять алгоритм работы с детектором языка - detector.language
    language = 'ru'
    morphy.append('ru')
    morphy.append('true' if blob[0].isupper() else 'false')
    for elem in tags:
      morphy.append(elem.normal_form) # allways .lower()
      morphy += define_morphy(pos = elem.tag.POS,
                              animacy = elem.tag.animacy, 
                              aspect = elem.tag.aspect,
                              case = elem.tag.case, 
                              gender = elem.tag.gender, 
                              involvement = elem.tag.involvement,
                              mood = elem.tag.mood, 
                              number = elem.tag.number,
                              person = elem.tag.person,
                              tense = elem.tag.tense,
                              transitivity = elem.tag.transitivity,
                              voice = elem.tag.voice)
  else:
    # en - ak da de gl hu la mt nl sco sl sm 
    if language != 'el':
      language = 'en'
    morphy.append(language)
    morphy.append('true' if blob[0].isupper() else 'false')
    morphy.append(textblob_lemmatize(blob).lower())
    pos, number = textblob_tags(blob)
    morphy += define_morphy(pos = pos, number = number)  
    if str(tags[0].tag)=='ROMN':
      morphy.append(tags[0].normal_form)
      morphy += define_morphy(pos = 'ROMN')

  return morphy

In [ ]:
'''
  Сервиcная функция:
  Загрузка выборки заданного количества фраз из файла 
'''
def load_xml(filename, paragraphs_amount):
    test_tree = ET.parse(filename)

    test_root = test_tree.getroot()
    test_body = test_root[0]
    test_row_count = paragraphs_amount + skip_amount
    return list(iter(test_body))[:test_row_count]

In [ ]:
'''
  Сервиcная функция:
  Вставка пробела в заданной позиции в строке
'''
def insert_space(text, index):
  return text[:index] + ' ' + text[index:]

In [ ]:
'''
  Основная функция для подготовки обучающей выборки.
  Создаём параллельные массивы с текстом — paragraphs,
  категориями лексем — tags
  и классификацией предложений — semantics
'''
def pars_xml(ps):
    # Массивы данных по параграфам
    paragraphs = []
    tags = []
    semantics = []
    exception_amount = 0
    exception_array = []
    # Массив фактических цветов в файле исходных данных
    colors = set() # все цвета в документе
    # длина массива лексем в параграфе
    max_paragraph_len = 0  # Максимальная длина предложения
    max_paragraph_len_ind = -1  # Индекс предложения с максимальной длиной

    for i, paragraph in enumerate(ps): # проходим по всем абзацам
        z_item = [0] # массив семантических индикаторов -  наличие определительной струткуры первого и второго уровня
        if i > colors_count - 1:     # пропускаем легенду для цветовой разметки    
            phrases_list = paragraph.findall(r) # находим все фразы (часть новости)
            p_words = [] # список сущностей в абзаце
            tags_w = [] # список семантических цветов для слов в абзаце

            last_ind = len(phrases_list)-1
            for ind, phrase in enumerate(phrases_list): # проходим по всем фразам в абзаце
                words = [] # список сущностей во фразе
                y_list = [] # категории для сущностей во фразе

                try:
                  text = phrase.find(t).text # получаем текст, содержащийся во фразе

                  # 1) выделяем пробелом точку (воскл.зн., вопр.зн., двоеточние) в конце предложения
                  text_comma = text.strip()
                  len_text = len(text_comma)-1
                  if ind==last_ind and len_text>0: 
                    if text_comma[len_text] in ['.', ':', '!', '?']:
                      text_comma = insert_space(text_comma, len_text)
                    
                  # 2) выделяем пробелами символы: , { } ( ) [ ] ; « » „ “ "
                  text_comma = text_comma.replace(',', ' , ')
                  text_comma = text_comma.replace('(', ' ( ')
                  text_comma = text_comma.replace(')', ' ) ')
                  text_comma = text_comma.replace('[', ' [ ')
                  text_comma = text_comma.replace(']', ' ] ')
                  text_comma = text_comma.replace('{', ' { ')
                  text_comma = text_comma.replace('}', ' } ')
                  text_comma = text_comma.replace('"', ' " ')
                  text_comma = text_comma.replace('„', ' „ ')
                  text_comma = text_comma.replace('“', ' “ ')
                  text_comma = text_comma.replace('«', ' « ')
                  text_comma = text_comma.replace('»', ' » ')
                  text_comma = text_comma.replace(';', ' ; ')
                  text_comma = text_comma.replace('%', ' %') # вставляем пробел именно слева, но не справа
                  text_comma = text_comma.replace('—', ' — ')
                  
            
                  # 3) заменяем все виды пробелов одиночным пробелом #32  
                  text_comma = text_comma.replace('\xa0', '')
                  text_comma = text_comma.replace('\x301', '')
                  text_comma = text_comma.replace('\u0301', '')
                  text_comma = text_comma.replace('\ufeff', '')
                  text_comma = text_comma.replace('  ', ' ')
                  text_comma = text_comma.replace('  ', ' ')
                  text_comma = text_comma.strip()      

                  # 4) выделяем левым пробелом двоеточие (:) при одновременном выполнении условий: 
                  # после двух буквенных символов в любом алфавите и перед правым пробелом. 
                  # https://habr.com/ru/post/349860/
                  text_comma = re.sub(r'(\w\w)(:)(\s)',  r'\1 \2\3', text_comma)
                  text_comma = re.sub(r'(\w\w)(:)$',  r'\1 \2', text_comma)

                  # отделяем пробелом слова в кириллице внутри угловых скобок — <ммм> 
                  text_comma = re.sub(r'(а..я)(>)',  r'\1 \2', text_comma)
                  text_comma = re.sub(r'(<)(а..я)',  r'\1 \2', text_comma)


                  if (len(text_comma)>0):
                    style = phrase.find(rpr) # получаем стили фразы
                    
                    if style.find(shd) is not None: # если размечали через заливку
                        color = style.find(shd).attrib[fill] # получаем значение цвета заливки
                    elif style.find(highlight) is not None: # если размечали через хайлайт
                        color = style.find(highlight).attrib[val] # получаем значение цвета хайлайта
                    else:
                        color = 'white' # иных вариантов выделения в word нет, значит эта фраза не выделена (белый цвет)
                    color = color.lower() # переводим строковое значение цвета в нижний регистр
                    meaning = color_meaning[color] if color in color_meaning else '' # если есть цвет в словаре цвет-значение, то получаем значение. в противном случае у фразы значение не было выделено
                    
                    colors.add(color) # добавляем цвет в словарь всех встреченных цветов, если нужно проанализировать их
                    
                    words = text_comma.split()
                    k = len(words)
                    if meaning in color_dic: # если во фразе присутствует какая-то выделяемая сущность
                      z_item = [1] # отмечаем фразу с понятийной структурой
                      for j in range(k):
                        y_list.append(meaning) # получаем индекс позиции, соответствующей какой-то семантической(смысловой) окраске и устанавливаем по этому индексу абсолютное значение категории
                    else:
                      for j in range(k):
                        y_list.append(zero_symbol)
            
                    p_words += words  # список сущностей в параграфе
                    tags_w += y_list # список семантических цветов для слов в параграфе

                except Exception:
                  exception_amount += 1
                  exception_array.append(i)

            # print(i, p_words)
            if p_words and len(p_words)>3: # количество слов в предложении позволяет иметь в себе определительную структуру 
                                           # как минимум из 3-х слов и завершающего знака препинания
              paragraphs.append(p_words)
              tags.append(tags_w)
              semantics.append(z_item) # записываем семантику
              
            if len(p_words) >= max_paragraph_len: 
              max_paragraph_len = len(p_words)
              max_paragraph_len_ind = i - skip_amount

        if i%100==0:
            print('line',i)

    return     paragraphs, tags, colors, max_paragraph_len, max_paragraph_len_ind, exception_array, semantics

In [ ]:
'''
  Графическая функция:
  Подсчёт и вывод на печать статистики о содержании категорий определительных структур в размеченных текстах
'''
def class_stat(paragraphs, tags, ext_print = False):
  classes = np.zeros((colors_count))
  len_tags = len(tags)
  print('Длина размеченной выборки:', len_tags)
  if ext_print:
    print("\nПеречень фраз со 100% разметкой:")
  for i, elem in enumerate(tags):
    if zero_symbol in elem:
      if len(set(elem))==1:
        classes[0] += 1
    else:
      if ext_print:
        print("paragraphs[{}]".format(i), '=',paragraphs[i])
        print("tags[{}]".format(i), '=',tags[i])
        print(" ")
    for item in list(color_dic.keys()):
      if item in elem:
        real_index = int(color_dic[item]) + 1
        classes[real_index] += 1
  print("\nСтатистика по классам")
  for i in range(len(classes)):
    if i==0:
      print(f'Фраз без определительных структур: {classes[0]} - {round(classes[0]*100./len_tags,2)}%')
    else:
      key=list(color_dic.keys())[list(color_dic.values()).index(i-1)] # ключ по значению
      print(f'Фраз с категорией {key}: {classes[i]} - {round(classes[i]*100./len_tags,2)}%')

In [ ]:
'''
  Графическая функция:
  Построение статистических графиков по входным данным.
'''
def draw_paragraphs_stat(paragraphs):
  # https://pyprog.pro/mpl/mpl_bar.html
  counter = []
  for elem in paragraphs:
    counter.append(len(elem))

  plt.figure(figsize=(27, 5))
  plt.grid(True)
  plt.bar(range(len(paragraphs)), counter)
  plt.suptitle('Фактическое распределение длин абзацев (в лексемах)')
  plt.show()

  dic_counter = collections.Counter(counter)
  print('Collections',dic_counter)

  plt.figure(figsize=(15, 5))
  plt.grid(True)
  plt.bar(dic_counter.keys(), dic_counter.values())
  plt.suptitle('Статистика кол-ва абзацев по их длинам')
  plt.show()

  return counter, dic_counter

In [ ]:
'''
  Сервисная функция:
  Сделать разметку текста цветом
'''
def tokens_to_text(x_arr, y_arr, max_len = 0):
  res = '' 
  str_len = 0
  def add_col(col, txt):
    nonlocal res, str_len
    if max_len > 0:
      next_len = len(txt)
      if str_len + next_len > max_len:
        res = res + col + '\n ' + txt
        str_len = next_len + 1
      else:
        res = res + col + ' ' + txt
        str_len += next_len + 1
    else:
      res = res + col + ' ' + txt

  for i, el in enumerate(x_arr):
    if el==0:
      break
    col_ind = y_arr[i].argmax() + 1
    add_col(tag_tokenizer_dic_color[col_ind], tokenizer.index_word[el])

  return res + color.END

In [ ]:
'''
  Сервисная функция:
  Сделать цветовую разметку текста
'''
def colorize_the_text(x_arr, y_arr, max_len = 0, padding = 'post', padding_len = 128):
  res = '' 
  str_len = 0
  def add_col(col, txt):
    nonlocal res, str_len
    if max_len > 0:
      next_len = len(txt)
      if str_len + next_len > max_len:
        res = res + col + '\n ' + txt
        str_len = next_len + 1
      else:
        res = res + col + ' ' + txt
        str_len += next_len + 1
    else:
      res = res + col + ' ' + txt

  start_y_arr = padding_len - len(x_arr)
  for i, el in enumerate(x_arr):
    if padding == 'post':
      col_ind = y_arr[i].argmax() + 1
    else:
      j = start_y_arr + i
      col_ind = y_arr[j].argmax() + 1
    add_col(tag_tokenizer_dic_color[col_ind], el)

  return res + color.END

In [ ]:
'''
  Сервисная функция для обучающей модели:
  Удалить неполные данные из массива с разметкой, полученной методом сегментации:
  — фразы без разметки;
  — один выделенный класс во фразе;
  — две лексемы на два выделенных класса.
'''
def cleansed_data(Ypred, Ytest, paragraphs):
  Ypred_res = []
  Ytest_res = []
  paragraphs_res = []
  delete = 0
  for i,phrase in enumerate(Ypred):
    uniq = set()
    all = []
    for elem in phrase:
      ind = np.argmax(elem)
      if ind>0:
        uniq.add(ind)
        all.append(ind)
    if len(uniq)>1 and len(all)>2:
      Ypred_res.append(phrase)
      Ytest_res.append(Ytest[i])
      paragraphs_res.append(paragraphs[i])
    else:
      delete += 1
  if delete>0:
    print(f'Очищено фраз: {delete}')
  return np.array(Ypred_res), np.array(Ytest_res), np.array(paragraphs_res)

In [ ]:
'''
  Сервисная функция для рабочей модели:
  Удалить неполные данные из массива с разметкой, полученной методом сегментации:
  — фразы без разметки;
  — один выделенный класс во фразе;
  — две лексемы на два выделенных класса;
  — фразы без "Термина" или "Дубль-Термина".
'''
def cleansed_work_data(Ypred, paragraphs):
  Ypred_res = []
  paragraphs_res = []
  delete = 0
  for i,phrase in enumerate(Ypred):
    uniq = set()
    all = []
    is_term = False
    for elem in phrase:
      ind = np.argmax(elem)
      if ind>0:
        uniq.add(ind)
        all.append(ind)
        if tag_tokenizer.index_word[ind + 1] in ['термин','дубль-термин']:
          is_term = True
    if len(uniq)>1 and len(all)>2 and is_term:
      Ypred_res.append(phrase)
      paragraphs_res.append(paragraphs[i])
    else:
      delete += 1
  if delete>0:
    print(f'Очищено фраз: {delete}')
  return np.array(Ypred_res), np.array(paragraphs_res)

# **Reading input data**

## Reading and verifying data (XML parsing). 

### Training sample

In [ ]:
paragraphs_amount = 3363
ps_train = load_xml('/content/drive/My Drive/Базы/NLP/База_словарей_train_6.xml', paragraphs_amount)

In [ ]:
paragraphs_train, tags_train, train_colors, max_paragraph_len, max_paragraph_len_ind, exception_array, semantics_train  = pars_xml(ps_train)
paragraphs_amount = len(paragraphs_train)  # обновляем длину массива на случай наличия в исходной базе пустых строк
paragraphs_amount, exception_array # контрольный вывод на размера принятых данных и массива индексов пустых строк

In [ ]:
# # Визуальный контроль правильности разбора файла
# for i, elem in enumerate(paragraphs_train):
#   print(i+1, elem)

In [ ]:
train_yes = 0
train_no = 0
for elem in semantics_train:
  if elem[0]==1:
    train_yes += 1
  else:
    train_no += 1
print(f"train: yes={train_yes}, no={train_no}")

In [ ]:
if len(train_colors) > colors_count:
  print('В документе цветов разметки больше допустимого')
# train_colors # выведем все цвета, которые мы встретили

In [ ]:
print(len(paragraphs_train))
print(len(tags_train))

print('Максимальная длина предложения — max_paragraph_len =',max_paragraph_len)
print('Индекс предложения с максимальной длиной — max_paragraph_len_ind =',max_paragraph_len_ind)

In [ ]:
# Корректируем максимальную длину предложения для использования U-net сеток
max_paragraph_len = 128

In [ ]:
train_paragraphs_len = len(paragraphs_train)
print('Длина обучающей выборки:', train_paragraphs_len)

**Извлекаем статистику по абзацам**

In [ ]:
train_counter, train_count = draw_paragraphs_stat(paragraphs_train)

**Во скольких фразах встречается тот или  иной класс**

In [ ]:
class_stat(paragraphs_train, tags_train)

### The extraction of morphological features for lexemes from the array paragraphs_train.

In [ ]:
morph_train = []
for i, elem in enumerate(paragraphs_train):
  row = []
  for j,lexem in enumerate(elem):
    row.append(morphyus(lexem))
  morph_train.append(row)
  # print(f'{i+1} done {j+1}!')

In [ ]:
# for mrph in morph_train[1999]:
#   print(mrph)

In [ ]:
# Статистика по длинам морфологических описаний для лексем 
k = 0
word = ''
index = -1
a = set()
for i, elem in enumerate(morph_train):
  for j,lexem in enumerate(elem):
    dlina = len(lexem)
    a.add(dlina)
    if dlina>k:
      index = i
      k = dlina
      word = lexem[2]

print(f'Лексема с максимально длинным морфологическим описанием: max(lexemdescription) = {k}; word = {word}; index(предложения) = {index}')
print('Перечень всех вариантов для длины морфологического описания лексемы\n',sorted(a))

In [ ]:
# Составим массив нормальных форм слов параллельный массиву paragraphs_train
# Составим массив частей речи параллельный массиву paragraphs_train
# Составим массив падежей слов параллельный массиву paragraphs_train
# Составим массив языковой принадлежности слов параллельный массиву paragraphs_train
norms_train = []
pos_train = []
case_train = []
lang_train = []
for i, elem in enumerate(morph_train):
  norms_sentence = []
  pos_sentence = []
  case_sentence = []
  lang_sentence = []
  for j,lexem in enumerate(elem):
    len_descr = len(lexem)
    lang_sentence.append(lexem[0])
    norms_sentence.append(lexem[2])
    # собрать все части речи у лексемы
    pos = set()
    for k in range(3, len_descr,13):
      pos.add(lexem[k])
    pos_sentence.append("".join(list(pos)))
    # собрать у лексемы все падежи и 
    # признаки формы глагола, причастия, деепричастия - GRND, PRTS, PRTF, INFN, VERB
    case = set()
    if lexem[3] in ['GRND', 'PRTS', 'PRTF', 'INFN', 'VERB']:
      for k in range(4, len_descr,13):
        for m in range(k, k+11):
          case.add(lexem[m] if not lexem[m] in ['None'] else '')
    else: 
      for k in range(6, len_descr,13):
        case.add(lexem[k])
    case_sentence.append("".join(list(case)))
  norms_train.append(norms_sentence)  
  pos_train.append(pos_sentence)
  case_train.append(case_sentence)
  lang_train.append(lang_sentence)

In [ ]:
# for i, elem in enumerate(lang_train):
#   print(i+1, elem)

In [ ]:
# for i, elem in enumerate(norms_train):
#   print(i+1, elem)

In [ ]:
# for i, elem in enumerate(pos_train):
#   print(i+1, elem)

In [ ]:
# for i, elem in enumerate(case_train):
#   print(i+1, elem)

### Testing sample

In [ ]:
test_paragraphs_amount = 703
ps_test = load_xml('/content/drive/My Drive/Базы/NLP/База_словарей_test_6.xml', test_paragraphs_amount)
print(len(ps_test))

In [ ]:
paragraphs_test, tags_test, test_colors, max_test_paragraph_len, max_test_paragraph_len_ind, test_exception_array, semantics_test  = pars_xml(ps_test)
test_paragraphs_amount = len(paragraphs_test)  # обновляем длину массива на случай наличия в исходной базе пустых строк
test_paragraphs_amount, test_exception_array # контрольный вывод на размера принятых данных и массива индексов пустых строк

In [ ]:
# # Визуальный контроль правильности разбора файла
# for i, elem in enumerate(paragraphs_test):
#   print(i+1, elem)

In [ ]:
test_yes = 0
test_no = 0
for elem in semantics_test:
  if elem[0]==1:
    test_yes += 1
  else:
    test_no += 1
print(f"train: yes={test_yes}, no={test_no}")

In [ ]:
if len(test_colors) > colors_count:
  print('В документе цветов разметки больше допустимого')
# test_colors # выведем все цвета, которые мы встретили

In [ ]:
print(len(paragraphs_test))
print(len(tags_test))

print('Максимальная длина предложения — max_test_paragraph_len =',max_test_paragraph_len)
print('Индекс предложения с максимальной длиной — max_test_paragraph_len_ind =',max_test_paragraph_len_ind)

In [ ]:
# Корректируем максимальную длину предложения для использования U-net сеток
max_test_paragraph_len = max_paragraph_len

**Извлекаем статистику по абзацам**

In [ ]:
test_counter, test_count = draw_paragraphs_stat(paragraphs_test)

**Во скольких фразах встречается тот или  иной класс**

In [ ]:
class_stat(paragraphs_test, tags_test)

### The extraction of morphological features for lexemes from the array paragraphs_test.

In [ ]:
morph_test = []
for i, elem in enumerate(paragraphs_test):
  row = []
  for j,lexem in enumerate(elem):
    row.append(morphyus(lexem))
  morph_test.append(row)
  # print(f'{i+1} done {j+1}!')

In [ ]:
len(morph_test)

In [ ]:
# for mrph in morph_test[665]:
#   print(mrph)

In [ ]:
# Статистика по длинам морфологических описаний для лексем 
k_test = 0
word_test = ''
index_test = -1
a_test = set()
for i, elem in enumerate(morph_test):
  for j,lexem in enumerate(elem):
    dlina = len(lexem)
    a.add(dlina)
    if dlina>k_test:
      index_test = i
      k_test = dlina
      word_test = lexem[2]

print(f'Лексема с максимально длинным морфологическим описанием: max(lexemdescription) = {k_test}; word = {word_test}; index(предложения) = {index_test}')
print('Перечень всех вариантов для длины морфологического описания лексемы\n',sorted(a))

In [ ]:
# Составим массив нормальных форм слов параллельный массиву paragraphs_test
# Составим массив частей речи параллельный массиву paragraphs_test
# Составим массив падежей слов параллельный массиву paragraphs_test
# Составим массив языковой принадлежности слов параллельный массиву paragraphs_test
norms_test = []
pos_test = []
case_test = []
lang_test = []
for i, elem in enumerate(morph_test):
  norms_sentence = []
  pos_sentence = []
  case_sentence = []
  lang_sentence = []
  for j,lexem in enumerate(elem):
    len_descr = len(lexem)
    lang_sentence.append(lexem[0])
    norms_sentence.append(lexem[2])
    # собрать все части речи у лексемы
    pos = set()
    for k in range(3, len_descr,13):
      pos.add(lexem[k])
    pos_sentence.append("".join(list(pos)))
    # собрать у лексемы все падежи и 
    # признаки формы глагола, причастия, деепричастия - GRND, PRTS, PRTF, INFN, VERB
    case = set()
    if lexem[3] in ['GRND', 'PRTS', 'PRTF', 'INFN', 'VERB']:
      for k in range(4, len_descr,13):
        for m in range(k, k+11):
          case.add(lexem[m] if not lexem[m] in ['None'] else '')
    else: 
      for k in range(6, len_descr,13):
        case.add(lexem[k])
    case_sentence.append("".join(list(case)))
  norms_test.append(norms_sentence)  
  pos_test.append(pos_sentence)
  case_test.append(case_sentence)
  lang_test.append(lang_sentence)

In [ ]:
# for i, elem in enumerate(lang_test):
#   print(i+1, elem)

In [ ]:
# for i, elem in enumerate(norms_test):
#   print(i+1, elem)

In [ ]:
# for i, elem in enumerate(pos_test):
#   print(i+1, elem)

In [ ]:
# for i, elem in enumerate(case_test):
#   print(i+1, elem)

# **Step 1. Text Classification** on the Parts of Speech

## Preparing a dataset

Объединим обучающий и тестовый массивы, чтобы воспользоваться общим токенизатором для нескольких входов с данными о pos, case, и т.д.

In [ ]:
full_pcln = pos_train + pos_test + case_train + case_test + lang_train + lang_test + norms_train + norms_test
full_semantics = semantics_train + semantics_test
print('Суммарная размерность входных данных:', len(full_pcln))
print('Размерность выходных данных:', len(full_semantics))

In [ ]:
full_pcln_together = [' '.join(sequence) for sequence in full_pcln]

In [ ]:
# full_pcln_together[max_paragraph_len_ind]

In [ ]:
# full_semantics[max_paragraph_len_ind]

In [ ]:
num_words = 10000 
# 2000 из расчёта: 
# 550 единиц на морфологические признаки и на наименования частей речи
# 1450 единиц на наиболее частотные лексемы из текста
sent_len = max_paragraph_len 
tokenizer = Tokenizer(num_words, filters=' \t\n', oov_token='<UNK>')

In [ ]:
tokenizer.fit_on_texts(full_pcln_together)

In [ ]:
# реальная длина словаря. Сравниваем со значением num_words
'реальная длина словаря токенизатора', len(tokenizer.index_word)

**Токенизируем входные данные.** (Выходные данные у нас изначально представляют собой токены)

In [ ]:
X = tokenizer.texts_to_sequences(full_pcln_together)

In [ ]:
Y = np.array(full_semantics)

**Теперь разобъём X на два входных массива с данными о pos, cases, lang и norms размерностью paragraphs_amount + test_paragraphs_amount каждый**

In [ ]:
L = paragraphs_amount + test_paragraphs_amount
M = 2*L
N = M+L
X_pos = X[:L]
X_case = X[L:M]
X_lang = X[M:N]
X_norms = X[N:]

**Проверка входящих и исходящих массивов на соответствие их размерностей** 

(для выявления факта порчи данных функцией Tokenizer)

In [ ]:
len(X_pos)==len(Y)

In [ ]:
len(X_case)==len(Y)

In [ ]:
len(X_lang)==len(Y)

In [ ]:
len(X_norms)==len(Y)

In [ ]:
# print(Y)

In [ ]:
# print(X_pos[0])

In [ ]:
# print(X_case[0])

In [ ]:
# print(X_lang[0])

In [ ]:
# print(X_norms[0])

**Выравнивание массивов до максимального размера абзаца**

In [ ]:
X_pos = pad_sequences(X_pos, sent_len, padding='post', truncating='post') # заполнение в конце и обрезка с конца предложения

In [ ]:
X_case = pad_sequences(X_case, sent_len, padding='post', truncating='post') # заполнение в конце и обрезка с конца предложения

In [ ]:
X_lang = pad_sequences(X_lang, sent_len, padding='post', truncating='post')

In [ ]:
X_norms = pad_sequences(X_norms, sent_len, padding='post', truncating='post')

In [ ]:
X_pos.shape

In [ ]:
X_case.shape

In [ ]:
X_lang.shape

In [ ]:
X_norms.shape

In [ ]:
Y.shape

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
Xtrain_pos, Xtest_pos, Ytrain, Ytest = train_test_split(X_pos, Y, train_size=train_paragraphs_len, shuffle=False) # Не перемешиваем данные и берём целую часть на тренировочную выборку

In [ ]:
Xtrain_case = X_case[:train_paragraphs_len]
Xtest_case = X_case[train_paragraphs_len:]

Xtrain_lang = X_lang[:train_paragraphs_len]
Xtest_lang = X_lang[train_paragraphs_len:]

Xtrain_norms = X_norms[:train_paragraphs_len]
Xtest_norms = X_norms[train_paragraphs_len:]

### Total for training and testing samples we have:

In [ ]:
print('Xtrain_pos.shape=',Xtrain_pos.shape)
print('Xtest_pos.shape=',Xtest_pos.shape)
print('Xtrain_case.shape=',Xtrain_case.shape)
print('Xtest_case.shape=',Xtest_case.shape)

print('Xtrain_lang.shape=',Xtrain_lang.shape)
print('Xtest_lang.shape=',Xtest_lang.shape)
print('Xtrain_norms.shape=',Xtrain_norms.shape)
print('Xtest_norms.shape=',Xtest_norms.shape)

print('Ytrain.shape=',Ytrain.shape)
print('Ytest.shape=',Ytest.shape)

In [ ]:
# Xtrain_pos[1]

In [ ]:
# Xtrain_case[1]

In [ ]:
# Xtrain_lang[1]

In [ ]:
# Xtrain_norms[1]

In [ ]:
# Ytrain[1]

## Recurrent neural network model (LSTM)

In [ ]:
embedding_size = 100

# параллельная архитектура
input_pos = Input((sent_len,))
x_pos = Embedding(num_words, embedding_size)(input_pos)
x_pos = Conv1D(64, 3, padding='same', name='input1_conv1')(x_pos)
x_pos = BatchNormalization()(x_pos)
x_pos = Activation('relu')(x_pos)
x_pos = Dropout(0.2)(x_pos)

input_case = Input((sent_len,))
x_case = Embedding(num_words, embedding_size)(input_case)
x_case = Conv1D(64, 3, padding='same', name='input2_conv1')(x_case)
x_case = BatchNormalization()(x_case)
x_case = Activation('relu')(x_case)
x_case = Dropout(0.2)(x_case)

input_lang = Input((sent_len,))
x_lang = Embedding(num_words, embedding_size)(input_lang)
x_lang = Conv1D(64, 3, padding='same', name='input3_conv1')(x_lang)
x_lang = BatchNormalization()(x_lang)
x_lang = Activation('relu')(x_lang)
x_lang = Dropout(0.2)(x_lang)

input_norms = Input((sent_len,))
x_norms = Embedding(num_words, embedding_size)(input_norms)
x_norms = Conv1D(64, 3, padding='same', name='input4_conv1')(x_norms)
x_norms = BatchNormalization()(x_norms)
x_norms = Activation('relu')(x_norms)
x_norms = Dropout(0.2)(x_norms)

x = concatenate([x_pos, x_case, x_lang, x_norms])

# последовательная архитектура
# x_pos = Embedding(num_words, embedding_size)(input_pos)
# x_pos = BatchNormalization()(x_pos)
# x_pos = LSTM(128, return_sequences=True)(x_pos)
# x_pos = Dropout(0.2)(x_pos)

# input_case = Input((sent_len,))
# x_case = Embedding(num_words, embedding_size)(input_case)

# input_lang = Input((sent_len,))
# x_lang = Embedding(num_words, embedding_size)(input_lang)

# input_norms = Input((sent_len,))
# x_norms = Embedding(num_words, embedding_size)(input_norms)


# x = concatenate([x_pos, x_case])
# x = BatchNormalization()(x)
# x = LSTM(128, return_sequences=True)(x)
# x = Dropout(0.2)(x)

# x = concatenate([x, x_lang])
# x = BatchNormalization()(x)
# x = LSTM(128, return_sequences=True)(x)
# x = Dropout(0.2)(x)

# x = concatenate([x, x_norms])
# x = BatchNormalization()(x)
x = LSTM(128, return_sequences=True)(x)
x = SpatialDropout1D(0.2)(x)
x = Flatten()(x)
# x = BatchNormalization()(x)
x = Dense(100, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(1, activation='sigmoid')(x)

modelLSTM = Model(inputs=[input_pos, input_case, input_lang, input_norms], outputs=output)

#  'adam' 'rmsprop'   Adam
modelLSTM.compile(loss='binary_crossentropy', metrics=[AUC(num_thresholds=150)], optimizer=RMSprop(lr=1e-4)) #

# modelLSTM.summary()

In [ ]:
'''
  callback-функция раннего останова по достижению достаточного процента точности
'''
def stop_NNT(about):
  print(about) 
  # https://www.tensorflow.org/guide/keras/custom_callback
  modelLSTM.stop_training = True

def on_epoch_end(epoch, logs):
  if (epoch > 10):
    precision = logs.get('val_auc')
    precision_train = logs.get('auc')
    if precision_train > 0.999 and precision > 0.996:
      stop_NNT("\nОбучение приостановлено при достаточном значении точности . val_auc = {}".format(round(precision, 3)))      

early_stop = tf.keras.callbacks.LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
plot_model(modelLSTM, show_shapes=True)

In [ ]:
historyLSTM = modelLSTM.fit(x=[Xtrain_pos, Xtrain_case, Xtrain_lang, Xtrain_norms], y=Ytrain, 
                            batch_size=96, epochs=61, 
                            validation_data=([Xtest_pos, Xtest_case, Xtest_lang, Xtest_norms], Ytest), 
                            callbacks=[early_stop])
                    
modelLSTM.save( '/content/drive/My Drive/tmp/modelLSTM_classification(production).h5' )

In [ ]:
# график прогрессии обучения нейросети в Matplotlib
plt.plot(historyLSTM.history['auc'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(historyLSTM.history['val_auc'], 
         label='Доля верных ответов на проверочном наборе')
plt.grid(True)
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend(loc='best')
plt.show()

**Проверка качества работы нейронной сети**

In [ ]:
Y_hat = modelLSTM.predict([Xtest_pos, Xtest_case, Xtest_lang, Xtest_norms])
'Размерность предсказаний', Y_hat.shape, 'Размерность тестовых данных', Ytest.shape

In [ ]:
y_coincidence = 0
y_unrecognized = 0
y_contrived = 0
for i, pred in enumerate(Y_hat):
  b = Ytest[i][0]
  a = round(pred[0])
  # print(i+1, a, b, paragraphs_test[i])
  if a+b>1.5:
    y_coincidence += 1
  elif a+b<0.5:
    y_coincidence += 1
  else:
    if a>0.5:
      y_contrived += 1
    else:
      y_unrecognized += 1

print(f"\nРаспознано: {y_coincidence}; Нераспознано: {y_unrecognized}; Надумано: {y_contrived}")  

ROC-кривая (англ. receiver operating characteristic, рабочая характеристика приёмника) — график, позволяющий оценить качество бинарной классификации, отображает соотношение между долей объектов от общего количества носителей признака, верно классифицированных как несущие признак (англ. true positive rate, TPR, называемой чувствительностью алгоритма классификации), и долей объектов от общего количества объектов, не несущих признака, ошибочно классифицированных как несущие признак (англ. false positive rate, FPR, величина 1-FPR называется специфичностью алгоритма классификации) при варьировании порога решающего правила.

https://ru.wikipedia.org/wiki/ROC-%D0%BA%D1%80%D0%B8%D0%B2%D0%B0%D1%8F

In [ ]:
plt.figure(figsize=(10,10))


num_classes = 1 #узнаем, сколько всего классов
for i in range(num_classes): #проходимся по всем классам
  fpr, tpr, thresholds = roc_curve(Ytest[:, i], Y_hat[:, i]) #получаем roc-кривую для очередного класса
  plt.plot(fpr, tpr, label='{} класс - {}, AUC={}'.format(i, dic_color[i], AUC()(Ytest[:, i], Y_hat[:, i]))) #отрисовываем кривую и подписываем ее площадь

avg_fpr, avg_tpr, thresholds = roc_curve(Ytest.flatten(), Y_hat.flatten()) #получаем micro-average (общую) кривую roc
plt.plot(avg_fpr, avg_tpr, 'mD',label='micro-average, AUC={}'.format(AUC()(Ytest, Y_hat)))#отрисовываем кривую и подписываем ее площадь
plt.plot([0,1], [0,1], 'b-.') #обозначим границу качественной работы нейросети
plt.fill_between([0,1], [0,1], label='Зона некачестванного распознавания', color='pink', alpha=0.3) #обозначим границу некачественной работы нейросети
plt.legend()

## Fetching arrays for the next step in extracting definitional structures in phrases with expected semantic status

In [ ]:
pos_train_2 = []
case_train_2 = []
lang_train_2 = []
norms_train_2 = []
tags_train_2 = []
paragraphs_train_2 = []
for i, elem in enumerate(semantics_train):
  if elem[0]==1:
    pos_train_2.append(pos_train[i])
    case_train_2.append(case_train[i])
    lang_train_2.append(lang_train[i])
    norms_train_2.append(norms_train[i])
    paragraphs_train_2.append(paragraphs_train[i])
    tags_train_2.append(tags_train[i])

print(f'Обучающие массивы (с размерностью {train_yes})', 
      len(pos_train_2), len(case_train_2), 
      len(lang_train_2), len(norms_train_2), 
      len(tags_train_2), len(paragraphs_train_2))


In [ ]:
pos_test_2 = []
case_test_2 = []
lang_test_2 = []
norms_test_2 = []
tags_test_2 = []
paragraphs_test_2 = []
for i, elem in enumerate(Y_hat):
  if round(elem[0])>0:
    pos_test_2.append(pos_test[i])
    case_test_2.append(case_test[i])
    lang_test_2.append(lang_test[i])
    norms_test_2.append(norms_test[i])
    paragraphs_test_2.append(paragraphs_test[i])
    tags_test_2.append(tags_test[i])

print(f'Тренировочные массивы (с размерностью {test_yes}):',
      len(pos_test_2), len(case_test_2), 
      len(lang_test_2), len(norms_test_2), 
      len(tags_test_2), len(paragraphs_test_2))


# **Step 2. Segmentation of text** on the Parts of Speech

## Preparing a dataset

Объединим обучающий и тестовый массивы, чтобы воспользоваться общим токенизатором.

In [ ]:
# Здесь для обучения оставим обучающую выборку в полном объёме, 
# а из тестовой возьмём только то, что выбрала нейронка на предыдущем шаге
all_pcln = pos_train + pos_test_2 + case_train + case_test_2 + \
           lang_train + lang_test_2 + norms_train + norms_test_2 
all_tags = tags_train + tags_test_2
all_paragraphs = paragraphs_train + paragraphs_test_2
train_test_paragraphs_amount_2 = len(all_paragraphs)

print('Суммарная размерность входных данных:', len(all_pcln))
print('Размерность выходных данных:', len(all_tags))
print('Размерность датасета:', len(all_paragraphs))

print(f'train_test_paragraphs_amount_2 = {train_test_paragraphs_amount_2}')

In [ ]:
all_pcln_together = [' '.join(sequence) for sequence in all_pcln]

In [ ]:
# len(all_pcln_together)

In [ ]:
# all_pcln_together[0]

In [ ]:
all_tags_together = [' '.join(tag) for tag in all_tags]

In [ ]:
# all_tags_together[0]

**Токенизируем входные и выходные данные.**

In [ ]:
X_2 = tokenizer.texts_to_sequences(all_pcln_together)

In [ ]:
Y_2 = tag_tokenizer.texts_to_sequences(all_tags_together)

In [ ]:
# Y_2[13]

**Теперь разобъём X на два входных массива с данными о pos и cases размерностью paragraphs_amount + test_paragraphs_amount каждый**

In [ ]:
L = train_test_paragraphs_amount_2
M = 2*L
N = M+L
X_2_pos = X_2[:L]
X_2_case = X_2[L:M]
X_2_lang = X_2[M:N]
X_2_norms = X_2[N:]

**Проверка входящих и исходящих массивов на соответствие их размерностей** 

(для выявления факта порчи данных функцией Tokenizer)

In [ ]:
len(X_2_pos)==len(Y_2)

In [ ]:
len(X_2_case)==len(Y_2)

In [ ]:
len(X_2_lang)==len(Y_2)

In [ ]:
len(X_2_norms)==len(Y_2)

In [ ]:
# Проверка на согласованность входных и выходных данных
for i, elem in enumerate(X_2_pos):
  if len(Y_2[i]) != len(X_2_pos[i]):
    print('X_2_pos[{}] ='.format(i),X_2_pos[i])
    print('Y_2[{}] ='.format(i),Y_2[i])
    # Если все массивы равны, то здесь  ничего не должно печататься
  else:
    pass
    # print('len(X_2_pos[{}]) ='.format(i),len(Y_2[i]))
  if len(Y_2[i]) != len(X_2_case[i]):
    print('X_2_case[{}] ='.format(i),X_2_case[i])
    print('Y_2[{}] ='.format(i),Y_2[i])
    # Если все массивы равны, то здесь  ничего не должно печататься
  else:
    pass
    # print('len(X_2_case[{}]) ='.format(i),len(Y_2[i]))
  if len(Y_2[i]) != len(X_2_lang[i]):
    print('X_2_lang[{}] ='.format(i),X_2_lang[i])
    print('Y_2[{}] ='.format(i),Y_2[i])
    # Если все массивы равны, то здесь  ничего не должно печататься
  else:
    pass
    # print('len(X_2_lang[{}]) ='.format(i),len(Y_2[i]))
  if len(Y_2[i]) != len(X_2_norms[i]):
    print('X_2_norms[{}] ='.format(i),X_2_norms[i])
    print('Y_2[{}] ='.format(i),Y_2[i])
    # Если все массивы равны, то здесь  ничего не должно печататься
  else:
    pass
    # print('len(X_2_norms[{}]) ='.format(i),len(Y_2[i]))

**Выравнивание массивов до максимального размера абзаца**

In [ ]:
X_2_pos = pad_sequences(X_2_pos, sent_len, padding='pre', truncating='post') # заполнение в начале и обрезка с конца предложения

In [ ]:
X_2_case = pad_sequences(X_2_case, sent_len, padding='pre', truncating='post') # заполнение в начале и обрезка с конца предложения

In [ ]:
X_2_lang = pad_sequences(X_2_lang, sent_len, padding='pre', truncating='post') # заполнение в начале и обрезка с конца предложения

In [ ]:
X_2_norms = pad_sequences(X_2_norms, sent_len, padding='pre', truncating='post') # заполнение в начале и обрезка с конца предложения

In [ ]:
X_2_pos.shape

In [ ]:
X_2_case.shape

In [ ]:
X_2_lang.shape

In [ ]:
X_2_norms.shape

In [ ]:
Y_2 = pad_sequences(Y_2, sent_len, value=1, padding='pre', truncating='post') # заполнение в начале и обрезка с конца предложения

In [ ]:
Y_2.shape

In [ ]:
Y_2 = to_categorical(Y_2, colors_count+1)  # см. лекцию "Занятие № 6 Углубленный курс по текстам" https://youtu.be/Nfw7ZY-WzS8?t=1533
Y_2.shape

In [ ]:
Y_2 = Y_2[:, :, 1:]  # см. лекцию "Занятие № 6 Углубленный курс по текстам" https://youtu.be/Nfw7ZY-WzS8?t=1533
Y_2.shape

In [ ]:
print(colorize_the_text(all_paragraphs[13], Y_2[13], 0, 'pre', sent_len))

Далее заменяем массивы из первого этапа по классификации предложений

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
Xtrain_pos, Xtest_pos, Ytrain, Ytest = train_test_split(X_2_pos, Y_2, train_size=train_paragraphs_len, shuffle=False) # Не перемешиваем данные и берём целую часть на тренировочную выборку

In [ ]:
Xtrain_case = X_2_case[:train_paragraphs_len]
Xtest_case = X_2_case[train_paragraphs_len:]

Xtrain_lang = X_2_lang[:train_paragraphs_len]
Xtest_lang = X_2_lang[train_paragraphs_len:]

Xtrain_norms = X_2_norms[:train_paragraphs_len]
Xtest_norms = X_2_norms[train_paragraphs_len:]

### Total for training and testing samples we have:

In [ ]:
# Новые размерности уже "только" для массивов с определительными структурами, 
# с учетом вычисленных на этапе классификации предложений

print('Xtrain_pos.shape=',Xtrain_pos.shape)
print('Xtest_pos.shape=',Xtest_pos.shape)
print('Xtrain_case.shape=',Xtrain_case.shape)
print('Xtest_case.shape=',Xtest_case.shape)
print('Xtrain_lang.shape=',Xtrain_lang.shape)
print('Xtest_lang.shape=',Xtest_lang.shape)
print('Xtrain_norms.shape=',Xtrain_norms.shape)
print('Xtest_norms.shape=',Xtest_norms.shape)
print('Ytrain.shape=',Ytrain.shape)
print('Ytest.shape=',Ytest.shape)

In [ ]:
# Xtrain_pos[1]

In [ ]:
# Xtrain_case[1]

In [ ]:
# Xtrain_lang[1]

In [ ]:
# Xtrain_norms[1]

In [ ]:
# Ytrain[1]

In [ ]:
# Пример окраски размеченного текста
print(colorize_the_text(paragraphs_test_2[0], Ytest[0], max_paragraph_len, 'pre', sent_len))

## Modelling

In [ ]:
emb_size = 128

In [ ]:
# '''
#   Собственная функция метрики, обрабатывающая пересечение двух областей
# '''
# def dice_coef(y_true, y_pred):
#     return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.) # Возвращаем площадь пересечения деленную на площадь объединения двух областей

In [ ]:
# model_dir = "/content/drive/My Drive/tmp/"

# '''
#   callback-функция сохранения на диск лучшей модели в процессе обучения
#   запись осуществляется на google-диск
# '''
# callbacks_list = [
#     tf.keras.callbacks.ModelCheckpoint(
#         filepath=os.path.join(model_dir, "weights" + "_epoch_{epoch}"),
#         monitor="loss",
#         save_best_only=True,
#         save_weights_only=True,
#         mode='min',
#         verbose=1
#     )
# ]

### **Recurrent neural network GRU**

#### Algorithm: Embedding + Bidirectional(GRU) + GRU , batch_size = 128, activation='softmax'

*Последовательная архитектура*

In [ ]:
input_pos = Input((sent_len,))
x_pos = Embedding(num_words, emb_size)(input_pos)
x_pos = BatchNormalization()(x_pos)
x_pos = Bidirectional(GRU(emb_size, return_sequences=True))(x_pos)
x_pos = Dropout(0.2)(x_pos)

input_case = Input((sent_len,))
x_case = Embedding(num_words, emb_size)(input_case)

input_lang = Input((sent_len,))
x_lang = Embedding(num_words, emb_size)(input_lang)

input_norms = Input((sent_len,))
x_norms = Embedding(num_words, emb_size)(input_norms)

x = concatenate([x_pos, x_case])
x = BatchNormalization()(x)
x = GRU(emb_size * 2, return_sequences=True)(x)
x = Dropout(0.2)(x)

x = concatenate([x, x_lang])
x = BatchNormalization()(x)
x = GRU(emb_size * 2, return_sequences=True)(x)
x = Dropout(0.2)(x)

x = concatenate([x, x_norms])
x = BatchNormalization()(x)
x = GRU(emb_size * 2, return_sequences=True)(x)
x = Dropout(0.2)(x)

output = Dense(colors_count, activation='softmax')(x)

modelEGRU = Model(inputs=[input_pos, input_case, input_lang, input_norms], outputs=output)

modelEGRU.compile(loss='categorical_crossentropy', optimizer='adam')
# modelEGRU.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=1e-4), metrics=[dice_coef, AUC(num_thresholds=150)])

plot_model(modelEGRU, show_shapes=True)

In [ ]:
historyEGRU = modelEGRU.fit(x=[Xtrain_pos, Xtrain_case, Xtrain_lang, Xtrain_norms], y=Ytrain,
                            epochs=100, batch_size=128,
                            validation_data=([Xtest_pos, Xtest_case, Xtest_lang, Xtest_norms], Ytest))
                    
modelEGRU.save( '/content/drive/My Drive/tmp/modelEGRU_consistent(production).h5' )

In [ ]:
# Отобразим график обучения модели
plt.figure(figsize=(14, 5))
plt.plot(historyEGRU.history['loss'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(historyEGRU.history['val_loss'], 
         label='Доля верных ответов на проверочном наборе')
plt.grid(True)
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

**Проверка тренировочной выборки (насколько хорошо выучены примеры)**

In [ ]:
# YpredEGRU_train = modelEGRU.predict([Xtrain_pos, Xtrain_case, Xtrain_lang, Xtrain_norms])

In [ ]:
# for i, el in enumerate(paragraphs_train):
#   print(f'{i+1}.')
#   print(colorize_the_text(el, Ytrain[i], 0, 'pre', sent_len))
#   print(colorize_the_text(el, YpredEGRU_train[i], 0, 'pre', sent_len))

*Чтобы память не забивать, результат 100%-го совпадения предсказаний с разметкой обучающей выборки удалён из текста*

**Проверка тестовой выборки (насколько хорошо обучена сеть)**

In [ ]:
YpredEGRU_test = modelEGRU.predict([Xtest_pos, Xtest_case, Xtest_lang, Xtest_norms])
YpredEGRU_test, Ytest_res, paragraphs_test_res = cleansed_data(YpredEGRU_test, Ytest, paragraphs_test_2) # нехорошие данные здесь всё-таки подчищаем

In [ ]:
# Встроенная функция sklearn возвращает и точность и полноту по классам
print(classification_report(Ytest_res.argmax(axis=-1).flatten(), 
                            YpredEGRU_test.argmax(axis=-1).flatten(), 
                            target_names=list(tag_tokenizer.index_word.values())))  

In [ ]:
# # выбираем конкретную фразу из тестовой выборки
# i = 254 - 1
# print(f'{i}.')
# print(colorize_the_text(paragraphs_test_res[i], Ytest_res[i], 0, 'pre', sent_len))
# print(colorize_the_text(paragraphs_test_res[i], YpredEGRU_test[i], 0, 'pre', sent_len))

In [ ]:
# # воспроизводим все фразы из тестовой выборки
# for i, el in enumerate(paragraphs_test_res):
#   print(f'{i+1}.')
#   print(colorize_the_text(el, Ytest_res[i], 0, 'pre', sent_len))
#   print(colorize_the_text(el, YpredEGRU_test[i], 0, 'pre', sent_len))

# # На github не отображается расцветка текста
# # Чтобы её посмотреть, воспользуйтесь ссылкой на https://colab.research.google.com/

#### Algorithm: Embedding + Bidirectional(GRU) + GRU , batch_size = 128, activation='softmax'

*Параллельная архитектура*

In [ ]:
# параллельная архитектура
input_pos = Input((sent_len,))
x_pos = Embedding(num_words, emb_size)(input_pos)
x_pos = BatchNormalization()(x_pos)
x_pos = Bidirectional(GRU(emb_size, return_sequences=True))(x_pos)
x_pos = Activation('relu')(x_pos)
x_pos = Dropout(0.2)(x_pos)

input_case = Input((sent_len,))
x_case = Embedding(num_words, embedding_size)(input_case)
x_case = BatchNormalization()(x_case)
x_case = Bidirectional(GRU(emb_size, return_sequences=True))(x_case)
x_case = Activation('relu')(x_case)
x_case = Dropout(0.2)(x_case)

input_lang = Input((sent_len,))
x_lang = Embedding(num_words, embedding_size)(input_lang)
x_lang = Bidirectional(GRU(emb_size, return_sequences=True))(x_lang)
x_lang = BatchNormalization()(x_lang)
x_lang = Activation('relu')(x_lang)
x_lang = Dropout(0.2)(x_lang)

input_norms = Input((sent_len,))
x_norms = Embedding(num_words, embedding_size)(input_norms)
x_norms = Bidirectional(GRU(emb_size, return_sequences=True))(x_norms)
x_norms = BatchNormalization()(x_norms)
x_norms = Activation('relu')(x_norms)
x_norms = Dropout(0.2)(x_norms)

x = concatenate([x_pos, x_case, x_lang, x_norms])

x = GRU(emb_size, return_sequences=True)(x)
# x = BatchNormalization()(x)
x = Dense(100, activation='relu')(x)
x = Dropout(0.2)(x)

output = Dense(colors_count, activation='softmax')(x)

modelEGRUpar = Model(inputs=[input_pos, input_case, input_lang, input_norms], outputs=output)

modelEGRUpar.compile(loss='categorical_crossentropy', optimizer='adam')
# modelEGRUpar.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=1e-4), metrics=[dice_coef, AUC(num_thresholds=150)])

plot_model(modelEGRUpar, show_shapes=True)

In [ ]:
historyEGRUpar = modelEGRUpar.fit(x=[Xtrain_pos, Xtrain_case, Xtrain_lang, Xtrain_norms], y=Ytrain,
                            epochs=100, batch_size=128,
                            validation_data=([Xtest_pos, Xtest_case, Xtest_lang, Xtest_norms], Ytest))
                    
modelEGRUpar.save( '/content/drive/My Drive/tmp/modelEGRU_parallel(production).h5' )

In [ ]:
# Отобразим график обучения модели
plt.figure(figsize=(14, 5))
plt.plot(historyEGRUpar.history['loss'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(historyEGRUpar.history['val_loss'], 
         label='Доля верных ответов на проверочном наборе')
plt.grid(True)
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

**Проверка тренировочной выборки (насколько хорошо выучены примеры)**

In [ ]:
# YpredEGRU_train_par = modelEGRUpar.predict([Xtrain_pos, Xtrain_case, Xtrain_lang, Xtrain_norms])

In [ ]:
# for i, el in enumerate(paragraphs_train):
#   print(f'{i+1}.')
#   print(colorize_the_text(el, Ytrain[i], 0, 'pre', sent_len))
#   print(colorize_the_text(el, YpredEGRU_train_par[i], 0, 'pre', sent_len))

*Чтобы память не забивать, результат 100%-го совпадения предсказаний с разметкой обучающей выборки удалён из текста*

**Проверка тестовой выборки (насколько хорошо обучена сеть)**

In [ ]:
YpredEGRU_test_par = modelEGRUpar.predict([Xtest_pos, Xtest_case, Xtest_lang, Xtest_norms])
YpredEGRU_test_par, Ytest_par_res, paragraphs_test_par_res = cleansed_data(YpredEGRU_test_par, Ytest, paragraphs_test_2) # нехорошие данные здесь всё-таки подчищаем

In [ ]:
# Встроенная функция sklearn возвращает и точность и полноту по классам 
print(classification_report(Ytest_par_res.argmax(axis=-1).flatten(), 
                            YpredEGRU_test_par.argmax(axis=-1).flatten(), 
                            target_names=list(tag_tokenizer.index_word.values())))  

In [ ]:
# # выбираем конкретную фразу из тестовой выборки
# i = 2 - 1
# print(f'{i}.')
# print(colorize_the_text(paragraphs_test_par_res[i], Ytest_par_res[i], 0, 'pre', sent_len))
# print(colorize_the_text(paragraphs_test_par_res[i], YpredEGRU_test_par[i], 0, 'pre', sent_len))

In [ ]:
# # воспроизводим все фразы из тестовой выборки
# for i, el in enumerate(paragraphs_test_par_res):
#   print(f'{i+1}.')
#   print(colorize_the_text(el, Ytest_par_res[i], 0, 'pre', sent_len))
#   print(colorize_the_text(el, YpredEGRU_test_par[i], 0, 'pre', sent_len))

# # На github не отображается расцветка текста
# # Чтобы её посмотреть, воспользуйтесь ссылкой на https://colab.research.google.com/

### **One-dimensional convolution**

#### Алгоритм: Embedding + Conv1D , batch_size = 16, activation='softmax'

*Последовательная архитектура*

In [ ]:
'''
  Функция создания сети
    Входные параметры:
    - num_classes - количество классов для рапознавания
    - sentence_len - длина предложения
    - num_words - количество слов в словаре
    - embedding_size - длина эмбединга
'''
def pseudoLinearSegmentationNet(
      num_classes = 9,
      sentence_len = 200,
      num_words = 5000,
      activation_function = 'softmax',
      embedding_size = 500
      ):
    input_pos = Input((sentence_len,))
    x_pos = Embedding(num_words, embedding_size)(input_pos)
    x_pos = Conv1D(64, 3, padding='same', name='input1_conv1')(x_pos)
    x_pos = BatchNormalization()(x_pos)
    x_pos = Activation('relu')(x_pos)

    input_case = Input((sentence_len,))
    x_case = Embedding(num_words, embedding_size)(input_case)

    input_lang = Input((sentence_len,))
    x_lang = Embedding(num_words, embedding_size)(input_lang)

    input_norms = Input((sentence_len,))
    x_norms = Embedding(num_words, embedding_size)(input_norms)

    x = concatenate([x_pos, x_case])
    x = Conv1D(128, 5, padding='same', name='input2_conv1')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, x_lang])
    x = Conv1D(128, 5, padding='same', name='input3_conv1')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, x_norms])
    x = Conv1D(128, 5, padding='same', name='input4_conv1')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = LSTM(128, return_sequences=True)(x)
    x = SpatialDropout1D(0.2)(x)
    # x = BatchNormalization()(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.2)(x)

    # x = Dropout(0.2)(x)
    output = Conv1D(num_classes,3, activation=activation_function, padding='same')(x)

    model = Model(inputs=[input_pos, input_case, input_lang, input_norms], outputs=output)

    model.compile(optimizer='rmsprop', 
                  loss='categorical_crossentropy')

    return model # Возвращаем сформированную модель

In [ ]:
modelL = pseudoLinearSegmentationNet(colors_count, sent_len, num_words, 'softmax', emb_size) # Создаем модель linearSegmentationNet
plot_model(modelL, show_shapes=True)

In [ ]:
historyL = modelL.fit(x=[Xtrain_pos, Xtrain_case, Xtrain_lang, Xtrain_norms], y=Ytrain, 
                      epochs=100, batch_size=64,
                      validation_data=([Xtest_pos, Xtest_case, Xtest_lang, Xtest_norms], Ytest)) # Обучаем модель на выборке по всем классам
modelL.save( '/content/drive/My Drive/tmp/modelL_consistent(production).h5' )

In [ ]:
# Отобразим график обучения модели
plt.figure(figsize=(14, 5))
plt.plot(historyL.history['loss'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(historyL.history['val_loss'], 
         label='Доля верных ответов на проверочном наборе')
plt.grid(True)
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

In [ ]:
YpredL_test = modelL.predict([Xtest_pos, Xtest_case, Xtest_lang, Xtest_norms])
YpredL_test, Ytest_res, paragraphs_test_res = cleansed_data(YpredL_test, Ytest, paragraphs_test_2) # нехорошие данные здесь всё-таки подчищаем

In [ ]:
# Встроенная функция sklearn возвращает и точность и полноту по классам
print(classification_report(Ytest_res.argmax(axis=-1).flatten(), 
                            YpredL_test.argmax(axis=-1).flatten(), 
                            target_names=list(tag_tokenizer.index_word.values())))  

**Проверка тестовой выборки (насколько хорошо обучена сеть)**

In [ ]:
# # выбираем конкретную фразу из тестовой выборки
# i = 2 - 1
# print(f'{i}.')
# print(colorize_the_text(paragraphs_test_res[i], Ytest_res[i], 0, 'pre', sent_len))
# print(colorize_the_text(paragraphs_test_res[i], YpredL_test[i], 0, 'pre', sent_len))

In [ ]:
# # воспроизводим все фразы из тестовой выборки
# for i, el in enumerate(paragraphs_test_res):
#   print(f'{i+1}.')
#   print(colorize_the_text(el, Ytest_res[i], 0, 'pre', sent_len))
#   print(colorize_the_text(el, YpredL_test[i], 0, 'pre', sent_len))

# # На github не отображается расцветка текста
# # Чтобы её посмотреть, воспользуйтесь ссылкой на https://colab.research.google.com/

#### Алгоритм: Embedding + Conv1D , batch_size = 16, activation='softmax'

*Параллельная архитектура*

In [ ]:
'''
  Функция создания сети
    Входные параметры:
    - num_classes - количество классов для рапознавания
    - sentence_len - длина предложения
    - num_words - количество слов в словаре
    - embedding_size - длина эмбединга
'''
def pseudoLinearSegmentationNet_par(
      num_classes = 9,
      sentence_len = 200,
      num_words = 5000,
      activation_function = 'softmax',
      embedding_size = 500
      ):
    input_pos = Input((sentence_len,))
    x_pos = Embedding(num_words, embedding_size)(input_pos)
    x_pos = Conv1D(64, 3, padding='same', name='input1_conv1')(x_pos)
    x_pos = BatchNormalization()(x_pos)
    x_pos = Activation('relu')(x_pos)
    x_pos = Dropout(0.2)(x_pos)

    input_case = Input((sentence_len,))
    x_case = Embedding(num_words, embedding_size)(input_case)
    x_case = Conv1D(64, 3, padding='same', name='input2_conv1')(x_case)
    x_case = BatchNormalization()(x_case)
    x_case = Activation('relu')(x_case)
    x_case = Dropout(0.2)(x_case)

    input_lang = Input((sentence_len,))
    x_lang = Embedding(num_words, embedding_size)(input_lang)
    x_lang = Conv1D(64, 3, padding='same', name='input3_conv1')(x_lang)
    x_lang = BatchNormalization()(x_lang)
    x_lang = Activation('relu')(x_lang)
    x_lang = Dropout(0.2)(x_lang)

    input_norms = Input((sentence_len,))
    x_norms = Embedding(num_words, embedding_size)(input_norms)
    x_norms = Conv1D(64, 3, padding='same', name='input4_conv1')(x_norms)
    x_norms = BatchNormalization()(x_norms)
    x_norms = Activation('relu')(x_norms)
    x_norms = Dropout(0.2)(x_norms)

    x = concatenate([x_pos, x_case, x_lang, x_norms])

    x = LSTM(128, return_sequences=True)(x)
    x = SpatialDropout1D(0.2)(x)
    # x = BatchNormalization()(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.2)(x)

    # x = Dropout(0.2)(x)
    output = Conv1D(num_classes,3, activation=activation_function, padding='same')(x)

    model = Model(inputs=[input_pos, input_case, input_lang, input_norms], outputs=output)

    model.compile(optimizer='rmsprop', 
                  loss='categorical_crossentropy')

    return model # Возвращаем сформированную модель

In [ ]:
modelL_par = pseudoLinearSegmentationNet_par(colors_count, sent_len, num_words, 'softmax', emb_size) # Создаем модель linearSegmentationNet
plot_model(modelL_par, show_shapes=True)

In [ ]:
historyL_par = modelL_par.fit(x=[Xtrain_pos, Xtrain_case, Xtrain_lang, Xtrain_norms], y=Ytrain, 
                      epochs=100, batch_size=64,
                      validation_data=([Xtest_pos, Xtest_case, Xtest_lang, Xtest_norms], Ytest)) # Обучаем модель на выборке по всем классам

modelL_par.save( '/content/drive/My Drive/tmp/modelL_parallel(production).h5' )

In [ ]:
# Отобразим график обучения модели
plt.figure(figsize=(14, 5))
plt.plot(historyL_par.history['loss'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(historyL_par.history['val_loss'], 
         label='Доля верных ответов на проверочном наборе')
plt.grid(True)
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

In [ ]:
YpredL_par = modelL_par.predict([Xtest_pos, Xtest_case, Xtest_lang, Xtest_norms])
YpredL_par, Ytest_par_res, paragraphs_par_test_res = cleansed_data(YpredL_par, Ytest, paragraphs_test_2) # нехорошие данные здесь всё-таки подчищаем


In [ ]:
# Встроенная функция sklearn возвращает и точность и полноту по классам
print(classification_report(Ytest_par_res.argmax(axis=-1).flatten(), 
                            YpredL_par.argmax(axis=-1).flatten(), 
                            target_names=list(tag_tokenizer.index_word.values())))  

**Проверка тестовой выборки (насколько хорошо обучена сеть)**

In [ ]:
# # выбираем конкретную фразу из тестовой выборки
# i = 2 - 1
# print(f'{i}.')
# print(colorize_the_text(paragraphs_par_test_res[i], Ytest_par_res[i], 0, 'pre', sent_len))
# print(colorize_the_text(paragraphs_par_test_res[i], YpredL_par[i], 0, 'pre', sent_len))

In [ ]:
# # воспроизводим все фразы из тестовой выборки
# for i, el in enumerate(paragraphs_par_test_res):
#   print(f'{i+1}.')
#   print(colorize_the_text(el, Ytest_par_res[i], 0, 'pre', sent_len))
#   print(colorize_the_text(el, YpredL_par[i], 0, 'pre', sent_len))

# # На github не отображается расцветка текста
# # Чтобы её посмотреть, воспользуйтесь ссылкой на https://colab.research.google.com/

### **U-net**

#### U-net classic

*Последовательная архитектура*



In [ ]:
'''
  Функция создания сети
    Входные параметры:
    - num_classes - количество классов
    - sentence_len - длина предложения
    - num_words - количество слов в словаре
    - embedding_size - длина эмбединга
'''

def unet(
      num_classes = 6,
      sentence_len = 200,
      num_words = 5000,
      activation_function = 'softmax',
      embedding_size = 50
      ):
    input_pos = Input((sentence_len,))
    x_pos = Embedding(num_words, embedding_size)(input_pos)
    x_pos = Conv1D(64, 3, padding='same', name='input1_conv1')(x_pos)
    x_pos = BatchNormalization()(x_pos)
    x_pos = Activation('relu')(x_pos)

    input_case = Input((sentence_len,))
    x_case = Embedding(num_words, embedding_size)(input_case)

    input_lang = Input((sentence_len,))
    x_lang = Embedding(num_words, embedding_size)(input_lang)

    input_norms = Input((sentence_len,))
    x_norms = Embedding(num_words, embedding_size)(input_norms)

    x = concatenate([x_pos, x_case])
    x = Conv1D(128, 5, padding='same', name='input2_conv1')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, x_lang])
    x = Conv1D(128, 5, padding='same', name='input3_conv1')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, x_norms])
    x = Conv1D(128, 5, padding='same', name='input4_conv1')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)


    # Block 1
    x = Conv1D(64, 3, padding='same', name='block1_conv1')(x) # Добавляем Conv1D-слой с 64-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = Conv1D(64, 3, padding='same', name='block1_conv2')(x)         # Добавляем Conv1D-слой с 64-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    block_1_out = Activation('relu')(x)                                    # Добавляем слой Activation и запоминаем в переменной block_1_out

    x = MaxPooling1D()(block_1_out)                                        # Добавляем слой MaxPooling1D

    # Block 2
    x = Conv1D(128, 3, padding='same', name='block2_conv1')(x)        # Добавляем Conv1D-слой с 128-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = Conv1D(128, 3, padding='same', name='block2_conv2')(x)        # Добавляем Conv1D-слой с 128-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    block_2_out = Activation('relu')(x)                                    # Добавляем слой Activation и запоминаем в переменной block_2_out

    x = MaxPooling1D()(block_2_out)                                        # Добавляем слой MaxPooling1D

    # Block 3
    x = Conv1D(256, 3, padding='same', name='block3_conv1')(x)        # Добавляем Conv1D-слой с 256-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = Conv1D(256, 3, padding='same', name='block3_conv2')(x)        # Добавляем Conv1D-слой с 256-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = Conv1D(256, 3, padding='same', name='block3_conv3')(x)        # Добавляем Conv1D-слой с 256-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    block_3_out = Activation('relu')(x)                                    # Добавляем слой Activation и запоминаем в переменной block_3_out

    x = MaxPooling1D()(block_3_out)                                        # Добавляем слой MaxPooling1D

    # Block 4
    x = Conv1D(512, 3, padding='same', name='block4_conv1')(x)        # Добавляем Conv1D-слой с 512-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = Conv1D(512, 3, padding='same', name='block4_conv2')(x)        # Добавляем Conv1D-слой с 256-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = Conv1D(512, 3, padding='same', name='block4_conv3')(x)        # Добавляем Conv1D-слой с 256-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    block_4_out = Activation('relu')(x)                                    # Добавляем слой Activation и запоминаем в переменной block_4_out
    x = block_4_out 

    # UP 2
    # x = Conv1DTranspose(x, 256, 2, strides=2, padding='same')(x)    # Добавляем слой Conv1DTranspose с 256 нейронами
    x = Lambda(lambda x: K.expand_dims(x, axis=2))(x)
    x = Conv2DTranspose(256, (2, 1), strides=(2, 1), padding='same')(x)
    x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = concatenate([x, block_3_out])                                      # Объединем текущий слой со слоем block_3_out
    x = Conv1D(256, 3, padding='same')(x)                             # Добавляем слой Conv1D с 256 нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = Conv1D(256, 3, padding='same')(x)
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    # UP 3
    # x = Conv1DTranspose(x, 128, 2, strides=2, padding='same')(x)    # Добавляем слой Conv1DTranspose с 128 нейронами
    x = Lambda(lambda x: K.expand_dims(x, axis=2))(x)
    x = Conv2DTranspose(128, (2, 1), strides=(2, 1), padding='same')(x)
    x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = concatenate([x, block_2_out])                                      # Объединем текущий слой со слоем block_2_out
    x = Conv1D(128, 3, padding='same')(x)                             # Добавляем слой Conv1D с 128 нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = Conv1D(128, 3, padding='same')(x) # Добавляем слой Conv1D с 128 нейронами
    x = BatchNormalization()(x) # Добавляем слой BatchNormalization
    x = Activation('relu')(x) # Добавляем слой Activation

    # UP 4
    # x = Conv1DTranspose(x, 64, 2, strides=2, padding='same')(x)    # Добавляем слой Conv1DTranspose с 64 нейронами
    x = Lambda(lambda x: K.expand_dims(x, axis=2))(x)
    x = Conv2DTranspose(64, (2, 1), strides=(2, 1), padding='same')(x)
    x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
    x = BatchNormalization()(x) # Добавляем слой BatchNormalization
    x = Activation('relu')(x) # Добавляем слой Activation

    x = concatenate([x, block_1_out])  # Объединем текущий слой со слоем block_1_out
    x = Conv1D(64, 3, padding='same')(x) # Добавляем слой Conv1D с 64 нейронами
    x = BatchNormalization()(x) # Добавляем слой BatchNormalization
    x = Activation('relu')(x) # Добавляем слой Activation

    x = Conv1D(64, 3, padding='same')(x) # Добавляем слой Conv1D с 64 нейронами
    x = BatchNormalization()(x) # Добавляем слой BatchNormalization
    x = Activation('relu')(x) # Добавляем слой Activation

    x = Conv1D(num_classes, 3, activation='softmax', padding='same')(x)  # Добавляем Conv1D-Слой с softmax-активацией на num_classes-нейронов

    model = Model([input_pos, input_case, input_lang, input_norms], x)                                   # Создаем модель со входами и выходом 'x'

    # Компилируем модель 
    model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy')
    
    return model # Возвращаем сформированную модель

In [ ]:
modelUnet = unet(colors_count, sent_len, num_words, 'softmax', emb_size) # Создаем модель unet
plot_model(modelUnet, show_shapes=True)

In [ ]:
historyUnet = modelUnet.fit(x=[Xtrain_pos, Xtrain_case, Xtrain_lang, Xtrain_norms], y=Ytrain, 
                            epochs=100, batch_size=64,
                            validation_data=([Xtest_pos, Xtest_case, Xtest_lang, Xtest_norms], Ytest))                    
modelUnet.save( '/content/drive/My Drive/tmp/modelUnet_consistent(production).h5' )

In [ ]:
# Отобразим график обучения модели
plt.figure(figsize=(14, 5))
plt.plot(historyUnet.history['loss'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(historyUnet.history['val_loss'], 
         label='Доля верных ответов на проверочном наборе')
plt.grid(True)
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

In [ ]:
YpredUnet = modelUnet.predict([Xtest_pos, Xtest_case, Xtest_lang, Xtest_norms])
YpredUnet_test, Ytest_res, paragraphs_test_res = cleansed_data(YpredUnet, Ytest, paragraphs_test_2) # нехорошие данные здесь всё-таки подчищаем

In [ ]:
# Встроенная функция sklearn возвращает и точность и полноту по классам
print(classification_report(Ytest_res.argmax(axis=-1).flatten(), 
                            YpredUnet_test.argmax(axis=-1).flatten(), 
                            target_names=list(tag_tokenizer.index_word.values())))  

**Проверка тестовой выборки (насколько хорошо обучена сеть)**

In [ ]:
# # выбираем конкретную фразу из тестовой выборки
# i = 2 
# print(f'{i + 1}.')
# print(colorize_the_text(paragraphs_test_res[i], Ytest_res[i], 0, 'pre', sent_len))
# print(colorize_the_text(paragraphs_test_res[i], YpredUnet_test[i], 0, 'pre', sent_len))

In [ ]:
# # воспроизводим все фразы из тестовой выборки
# for i, el in enumerate(paragraphs_test_res):
#   print(f'{i+1}.')
#   print(colorize_the_text(el, Ytest_res[i], 0, 'pre', sent_len))
#   print(colorize_the_text(el, YpredUnet_test[i], 0, 'pre', sent_len))

# # На github не отображается расцветка текста
# # Чтобы её посмотреть, воспользуйтесь ссылкой на https://colab.research.google.com/ выше

#### U-net classic

*Параллельная архитектура*



In [ ]:
'''
  Функция создания сети
    Входные параметры:
    - num_classes - количество классов
    - sentence_len - длина предложения
    - num_words - количество слов в словаре
    - embedding_size - длина эмбединга
'''

def unet_par(
      num_classes = 6,
      sentence_len = 200,
      num_words = 5000,
      activation_function = 'softmax',
      embedding_size = 50
      ):
    # параллельная архитектура
    input_pos = Input((sentence_len,))
    x_pos = Embedding(num_words, embedding_size)(input_pos)
    x_pos = BatchNormalization()(x_pos)
    x_pos = Conv1D(64, 3, padding='same', name='input1_conv1')(x_pos)
    x_pos = Activation('relu')(x_pos)
    x_pos = Dropout(0.2)(x_pos)

    input_case = Input((sentence_len,))
    x_case = Embedding(num_words, embedding_size)(input_case)
    x_case = BatchNormalization()(x_case)
    x_case = Conv1D(64, 3, padding='same', name='input2_conv1')(x_case)
    x_case = Activation('relu')(x_case)
    x_case = Dropout(0.2)(x_case)

    input_lang = Input((sentence_len,))
    x_lang = Embedding(num_words, embedding_size)(input_lang)
    x_lang = Conv1D(64, 3, padding='same', name='input3_conv1')(x_lang)
    x_lang = BatchNormalization()(x_lang)
    x_lang = Activation('relu')(x_lang)
    x_lang = Dropout(0.2)(x_lang)

    input_norms = Input((sentence_len,))
    x_norms = Embedding(num_words, embedding_size)(input_norms)
    x_norms = Conv1D(64, 3, padding='same', name='input4_conv1')(x_norms)
    x_norms = BatchNormalization()(x_norms)
    x_norms = Activation('relu')(x_norms)
    x_norms = Dropout(0.2)(x_norms)

    x = concatenate([x_pos, x_case, x_lang, x_norms])


    # Block 1
    x = Conv1D(64, 3, padding='same', name='block1_conv1')(x) # Добавляем Conv1D-слой с 64-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = Conv1D(64, 3, padding='same', name='block1_conv2')(x)         # Добавляем Conv1D-слой с 64-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    block_1_out = Activation('relu')(x)                                    # Добавляем слой Activation и запоминаем в переменной block_1_out

    x = MaxPooling1D()(block_1_out)                                        # Добавляем слой MaxPooling1D

    # Block 2
    x = Conv1D(128, 3, padding='same', name='block2_conv1')(x)        # Добавляем Conv1D-слой с 128-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = Conv1D(128, 3, padding='same', name='block2_conv2')(x)        # Добавляем Conv1D-слой с 128-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    block_2_out = Activation('relu')(x)                                    # Добавляем слой Activation и запоминаем в переменной block_2_out

    x = MaxPooling1D()(block_2_out)                                        # Добавляем слой MaxPooling1D

    # Block 3
    x = Conv1D(256, 3, padding='same', name='block3_conv1')(x)        # Добавляем Conv1D-слой с 256-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = Conv1D(256, 3, padding='same', name='block3_conv2')(x)        # Добавляем Conv1D-слой с 256-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = Conv1D(256, 3, padding='same', name='block3_conv3')(x)        # Добавляем Conv1D-слой с 256-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    block_3_out = Activation('relu')(x)                                    # Добавляем слой Activation и запоминаем в переменной block_3_out

    x = MaxPooling1D()(block_3_out)                                        # Добавляем слой MaxPooling1D

    # Block 4
    x = Conv1D(512, 3, padding='same', name='block4_conv1')(x)        # Добавляем Conv1D-слой с 512-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = Conv1D(512, 3, padding='same', name='block4_conv2')(x)        # Добавляем Conv1D-слой с 256-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = Conv1D(512, 3, padding='same', name='block4_conv3')(x)        # Добавляем Conv1D-слой с 256-нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    block_4_out = Activation('relu')(x)                                    # Добавляем слой Activation и запоминаем в переменной block_4_out
    x = block_4_out 

    # UP 2
    # x = Conv1DTranspose(x, 256, 2, strides=2, padding='same')(x)    # Добавляем слой Conv1DTranspose с 256 нейронами
    x = Lambda(lambda x: K.expand_dims(x, axis=2))(x)
    x = Conv2DTranspose(256, (2, 1), strides=(2, 1), padding='same')(x)
    x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = concatenate([x, block_3_out])                                      # Объединем текущий слой со слоем block_3_out
    x = Conv1D(256, 3, padding='same')(x)                             # Добавляем слой Conv1D с 256 нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = Conv1D(256, 3, padding='same')(x)
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    # UP 3
    # x = Conv1DTranspose(x, 128, 2, strides=2, padding='same')(x)    # Добавляем слой Conv1DTranspose с 128 нейронами
    x = Lambda(lambda x: K.expand_dims(x, axis=2))(x)
    x = Conv2DTranspose(128, (2, 1), strides=(2, 1), padding='same')(x)
    x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = concatenate([x, block_2_out])                                      # Объединем текущий слой со слоем block_2_out
    x = Conv1D(128, 3, padding='same')(x)                             # Добавляем слой Conv1D с 128 нейронами
    x = BatchNormalization()(x)                                            # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                              # Добавляем слой Activation

    x = Conv1D(128, 3, padding='same')(x) # Добавляем слой Conv1D с 128 нейронами
    x = BatchNormalization()(x) # Добавляем слой BatchNormalization
    x = Activation('relu')(x) # Добавляем слой Activation

    # UP 4
    # x = Conv1DTranspose(x, 64, 2, strides=2, padding='same')(x)    # Добавляем слой Conv1DTranspose с 64 нейронами
    x = Lambda(lambda x: K.expand_dims(x, axis=2))(x)
    x = Conv2DTranspose(64, (2, 1), strides=(2, 1), padding='same')(x)
    x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
    x = BatchNormalization()(x) # Добавляем слой BatchNormalization
    x = Activation('relu')(x) # Добавляем слой Activation

    x = concatenate([x, block_1_out])  # Объединем текущий слой со слоем block_1_out
    x = Conv1D(64, 3, padding='same')(x) # Добавляем слой Conv1D с 64 нейронами
    x = BatchNormalization()(x) # Добавляем слой BatchNormalization
    x = Activation('relu')(x) # Добавляем слой Activation

    x = Conv1D(64, 3, padding='same')(x) # Добавляем слой Conv1D с 64 нейронами
    x = BatchNormalization()(x) # Добавляем слой BatchNormalization
    x = Activation('relu')(x) # Добавляем слой Activation

    x = Conv1D(num_classes, 3, activation='softmax', padding='same')(x)  # Добавляем Conv1D-Слой с softmax-активацией на num_classes-нейронов

    model = Model([input_pos, input_case, input_lang, input_norms], x)                                   # Создаем модель со входами и выходом 'x'

    # Компилируем модель 
    model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy')
    
    return model # Возвращаем сформированную модель

In [ ]:
modelUnet_par = unet_par(colors_count, sent_len, num_words, 'softmax', emb_size) # Создаем модель unet
plot_model(modelUnet_par, show_shapes=True)

In [ ]:
historyUnet_par = modelUnet_par.fit(x=[Xtrain_pos, Xtrain_case, Xtrain_lang, Xtrain_norms], y=Ytrain, 
                            epochs=100, batch_size=64,
                            validation_data=([Xtest_pos, Xtest_case, Xtest_lang, Xtest_norms], Ytest))                    
modelUnet.save( '/content/drive/My Drive/tmp/modelUnet_parallel(production).h5' )

In [ ]:
# Отобразим график обучения модели
plt.figure(figsize=(14, 5))
plt.plot(historyUnet_par.history['loss'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(historyUnet_par.history['val_loss'], 
         label='Доля верных ответов на проверочном наборе')
plt.grid(True)
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

In [ ]:
YpredUnet_par = modelUnet_par.predict([Xtest_pos, Xtest_case, Xtest_lang, Xtest_norms])
YpredUnet_par, Ytest_res_par, paragraphs_test_res_par = cleansed_data(YpredUnet_par, Ytest, paragraphs_test_2) # нехорошие данные здесь всё-таки подчищаем

In [ ]:
# Встроенная функция sklearn возвращает и точность и полноту по классам
print(classification_report(Ytest_res_par.argmax(axis=-1).flatten(), 
                            YpredUnet_par.argmax(axis=-1).flatten(), 
                            target_names=list(tag_tokenizer.index_word.values())))  

**Проверка тестовой выборки (насколько хорошо обучена сеть)**

In [ ]:
# # выбираем конкретную фразу из тестовой выборки
# i = 2
# print(f'{i + 1}.')
# print(colorize_the_text(paragraphs_test_res_par[i], Ytest_res_par[i], 0, 'pre', sent_len))
# print(colorize_the_text(paragraphs_test_res_par[i], YpredUnet_par[i], 0, 'pre', sent_len))

In [ ]:
# # воспроизводим все фразы из тестовой выборки
# for i, el in enumerate(paragraphs_test_res_par):
#   print(f'{i+1}.')
#   print(colorize_the_text(el, Ytest_res_par[i], 0, 'pre', sent_len))
#   print(colorize_the_text(el, YpredUnet_par[i], 0, 'pre', sent_len))

# # На github не отображается расцветка текста
# # Чтобы её посмотреть, воспользуйтесь ссылкой на https://colab.research.google.com/ выше

# **Checking the neural network model with new data**

In [ ]:
input_file = '/content/drive/My Drive/Базы/NLP/web.kpi.kharkov.ua.pomada.txt'  # http://web.kpi.kharkov.ua/nanochem/sostav-gubnoj-pomady-infografika/
# input_file = '/content/drive/My Drive/Базы/NLP/proproprogs.ru.descriptors.txt' # https://proproprogs.ru/python_oop/sozdanie-deskriptorov-klassov

# model_file = '/content/drive/My Drive/tmp/modelEGRU_consistent(production).h5'
# model_file = '/content/drive/My Drive/tmp/modelEGRU_parallel(production).h5'
model_file = '/content/drive/My Drive/tmp/modelL_consistent(production).h5'
# model_file = '/content/drive/My Drive/tmp/modelL_parallel(production).h5'
# model_file = '/content/drive/My Drive/tmp/modelUnet_consistent(production).h5'
# model_file = '/content/drive/My Drive/tmp/modelUnet_parallel(production).h5'

## Input data

In [ ]:
print('Читаем данные из файла с google-диска')
f = open(input_file, 'r') #Открываем наш файл для чтения и считываем из него данные 
text = f.read() #Записываем прочитанный текст в переменную 

text = re.sub(r'\n\n',  r'\n', text)
text = re.sub(r'\n\n',  r'\n', text)
text = re.sub(r'(\n)([А-Я])',  r'. \2', text)
text = re.sub(r'\n',  r' ', text)
text = text.replace('..',  '.')

txt_list = sent_tokenize(text, language="russian")

print(f'Длина входного массива (фраз): {len(txt_list)}')

paragraphs_work = []
max_work_paragraph_len = 0
for elem in txt_list: 
  lexems = word_tokenize(elem, language="russian")
  paragraph_length = len(lexems)
  max_work_paragraph_len = paragraph_length if paragraph_length>max_work_paragraph_len else max_work_paragraph_len
  if paragraph_length>3 and paragraph_length<129:
    paragraphs_work.append(lexems) 
  # print(lexems)
work_paragraphs_amount = len(paragraphs_work)
print('work_paragraphs_amount',work_paragraphs_amount, 'max_work_paragraph_len',max_work_paragraph_len)

print('Извлекаем морфологические признаки.')
morph_work = []
for i, elem in enumerate(paragraphs_work):
  row = []
  for j,lexem in enumerate(elem):
    row.append(morphyus(lexem))
  morph_work.append(row)
  print(f'{i+1} done {j+1}!')
print(f'Длина массива с морфологическими данными: {len(morph_work)}')

# Составим массив нормальных форм слов параллельный массиву paragraphs_work
# Составим массив частей речи параллельный массиву paragraphs_work
# Составим массив падежей слов параллельный массиву paragraphs_work
# Составим массив языковой принадлежности слов параллельный массиву paragraphs_work
norms_work = []
pos_work = []
case_work = []
lang_work = []
for i, elem in enumerate(morph_work):
  norms_sentence = []
  pos_sentence = []
  case_sentence = []
  lang_sentence = []
  for j,lexem in enumerate(elem):
    len_descr = len(lexem)
    lang_sentence.append(lexem[0])
    norms_sentence.append(lexem[2])
    # собрать все части речи у лексемы
    pos = set()
    for k in range(3, len_descr,13):
      pos.add(lexem[k])
    pos_sentence.append("".join(list(pos)))
    # собрать у лексемы все падежи и 
    # признаки формы глагола, причастия, деепричастия - GRND, PRTS, PRTF, INFN, VERB
    case = set()
    if lexem[3] in ['GRND', 'PRTS', 'PRTF', 'INFN', 'VERB']:
      for k in range(4, len_descr,13):
        for m in range(k, k+11):
          case.add(lexem[m] if not lexem[m] in ['None'] else '')
    else: 
      for k in range(6, len_descr,13):
        case.add(lexem[k])
    case_sentence.append("".join(list(case)))
  norms_work.append(norms_sentence)  
  pos_work.append(pos_sentence)
  case_work.append(case_sentence)
  lang_work.append(lang_sentence)



## Dataset

In [ ]:
print('Собираем входные данные в один массив.')
full_work_pcln = pos_work + case_work + lang_work + norms_work
full_work_pcln_together = [' '.join(sequence) for sequence in full_work_pcln]

print('Токенизируем входные данные.')
wX = tokenizer.texts_to_sequences(full_work_pcln_together)

print('Разбиваем wX на входные массивы по типам данных: pos, cases, lang и norms.')
L = work_paragraphs_amount
M = 2*L
N = M+L
wX_pos = wX[:L]
wX_case = wX[L:M]
wX_lang = wX[M:N]
wX_norms = wX[N:]

print('Выравниваем фразы из токенов до максимального размера абзаца.')
wX_pos = pad_sequences(wX_pos, sent_len, padding='pre', truncating='post')  # заполнение в начале и обрезка с конца предложения
wX_case = pad_sequences(wX_case, sent_len, padding='pre', truncating='post')
wX_lang = pad_sequences(wX_lang, sent_len, padding='pre', truncating='post')
wX_norms = pad_sequences(wX_norms, sent_len, padding='pre', truncating='post')

print('Сводная информация по подготовленным входным массивам.')
print('wX_pos.shape=',wX_pos.shape)
print('wX_case.shape=',wX_case.shape)
print('wX_lang.shape=',wX_lang.shape)
print('wX_norms.shape=',wX_norms.shape)

## Segmentation

http://web.kpi.kharkov.ua/nanochem/sostav-gubnoj-pomady-infografika/

/content/drive/My Drive/Базы/NLP/web.kpi.kharkov.ua.pomada.txt

In [ ]:
print('Подключаемся к нейросетевой модели.')
# # параллельная архитектура
# input_pos = Input((sent_len,))
# x_pos = Embedding(num_words, emb_size)(input_pos)
# x_pos = BatchNormalization()(x_pos)
# x_pos = Bidirectional(GRU(emb_size, return_sequences=True))(x_pos)
# x_pos = Activation('relu')(x_pos)
# x_pos = Dropout(0.2)(x_pos)

# input_case = Input((sent_len,))
# x_case = Embedding(num_words, emb_size)(input_case)
# x_case = BatchNormalization()(x_case)
# x_case = Bidirectional(GRU(emb_size, return_sequences=True))(x_case)
# x_case = Activation('relu')(x_case)
# x_case = Dropout(0.2)(x_case)

# input_lang = Input((sent_len,))
# x_lang = Embedding(num_words, emb_size)(input_lang)
# x_lang = Bidirectional(GRU(emb_size, return_sequences=True))(x_lang)
# x_lang = BatchNormalization()(x_lang)
# x_lang = Activation('relu')(x_lang)
# x_lang = Dropout(0.2)(x_lang)

# input_norms = Input((sent_len,))
# x_norms = Embedding(num_words, emb_size)(input_norms)
# x_norms = Bidirectional(GRU(emb_size, return_sequences=True))(x_norms)
# x_norms = BatchNormalization()(x_norms)
# x_norms = Activation('relu')(x_norms)
# x_norms = Dropout(0.2)(x_norms)

# x = concatenate([x_pos, x_case, x_lang, x_norms])

# x = GRU(emb_size, return_sequences=True)(x)
# x = Dense(100, activation='relu')(x)
# x = Dropout(0.2)(x)

# output = Dense(colors_count, activation='softmax')(x)

# modelEGRUpar = Model(inputs=[input_pos, input_case, input_lang, input_norms], outputs=output)

# modelEGRUpar.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[dice_coef])

model_cur = load_model(model_file)

print('\nЗапуск сегментации текста.')
Ypred_cur = model_cur.predict([wX_pos, wX_case, wX_lang, wX_norms])
Ypred_cur_res, paragraphs_work_res = cleansed_work_data(Ypred_cur, paragraphs_work) # подчищаем нехорошие данные

print('\nРезультат сегментации текста.')
for i, el in enumerate(paragraphs_work_res):
  print(f'{i+1}.')
  print(colorize_the_text(el, Ypred_cur_res[i], 0, 'pre', sent_len))